In [1]:
# Kaggle cell: resume training + push checkpoints to HF
!pip -q install -U transformers datasets accelerate huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 76.9 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.7/374.7 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 27.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 95.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 60.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 

In [6]:
import os, math
from pathlib import Path
from kaggle_secrets import UserSecretsClient
from huggingface_hub import login, snapshot_download
from datasets import load_dataset
from transformers import (
    RobertaForMaskedLM, RobertaTokenizerFast,
    DataCollatorForLanguageModeling, Trainer, TrainingArguments
)

In [7]:
repo128 = "mhurhangee/patroberta-mlm-sl128-v8000"
repo256 = "mhurhangee/patroberta-mlm-sl256-v8000"

In [8]:
model = RobertaForMaskedLM.from_pretrained(repo128) 

config.json:   0%|          | 0.00/633 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.05M [00:00<?, ?B/s]

In [9]:
model.gradient_checkpointing_disable()

In [10]:
tok = RobertaTokenizerFast.from_pretrained(repo128)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/965 [00:00<?, ?B/s]

In [11]:
ds = load_dataset("mhurhangee/ep-patent-all-claims")
train_ds, val_ds = ds["train"], ds["validation"]

README.md:   0%|          | 0.00/385 [00:00<?, ?B/s]

data/train-00000-of-00006.parquet:   0%|          | 0.00/147M [00:00<?, ?B/s]

data/train-00001-of-00006.parquet:   0%|          | 0.00/147M [00:00<?, ?B/s]

data/train-00002-of-00006.parquet:   0%|          | 0.00/147M [00:00<?, ?B/s]

data/train-00003-of-00006.parquet:   0%|          | 0.00/147M [00:00<?, ?B/s]

...

data/validation-00000-of-00001.parquet:   0%|          | 0.00/18.5M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/18.5M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4243904 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/88415 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/88415 [00:00<?, ? examples/s]

In [15]:
seq_len = 256
def enc(b): return tok(b["text"], truncation=True, max_length=seq_len)
train_ds = train_ds.map(enc, batched=True, remove_columns=train_ds.column_names)
val_ds   = val_ds.map(enc,   batched=True, remove_columns=val_ds.column_names)

Map:   0%|          | 0/4243904 [00:00<?, ? examples/s]

Map:   0%|          | 0/88415 [00:00<?, ? examples/s]

In [16]:
collator = DataCollatorForLanguageModeling(tokenizer=tok, mlm=True, mlm_probability=0.15)

In [23]:
from kaggle_secrets import UserSecretsClient
hf_token = UserSecretsClient().get_secret("HF_TOKEN")
os.environ["HUGGING_FACE_HUB_TOKEN"] = hf_token

In [24]:
args = TrainingArguments(
    output_dir="./checkpoints",
    per_device_train_batch_size=128,
    per_device_eval_batch_size=32,
    gradient_accumulation_steps=1,
    learning_rate=4e-4,
    weight_decay=0.01,
    warmup_ratio=0.06,
    num_train_epochs=4,
    eval_strategy="steps",
    eval_steps=1000,
    save_strategy="steps",
    save_steps=1000,
    save_total_limit=4,
    logging_steps=100,
    fp16=True,
    fp16_full_eval=True,
    report_to="none",
    load_best_model_at_end=True,
    metric_for_best_model="loss",
    greater_is_better=False,
    remove_unused_columns=False,
    # Hub pushing
    push_to_hub=True,
    hub_model_id=repo256,
    hub_token=hf_token,
    hub_private_repo=False,
    hub_strategy="every_save",   # push each checkpoint on save
)

In [25]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    processing_class=tok,
    data_collator=collator,

)

In [ ]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Step,Training Loss,Validation Loss
1000,2.773400,2.535119
2000,2.271400,1.890647
3000,2.148100,1.867113
4000,2.155600,1.877035
5000,2.154700,1.869903
6000,2.145800,1.868883
7000,2.149900,1.868450
8000,2.140100,1.862905
9000,2.147100,1.858026
10000,2.131400,1.857904


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked t

In [ ]:
trainer.save_model()
tokenizer.save_pretrained(args.output_dir)
trainer.push_to_hub()    